# Lab 02: LSTM - exercise

In [1]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # find automatically the path of the folder containing "file_name" :
    file_name = 'lstm_exercise.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    #path_to_file = '/content/gdrive/My Drive/AI6103_2020_codes/codes/labs_lecture10/lab02_lstm'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd
    

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 274 sec ( 4.56 min)<br> 
* Time for 1 epoch on GPU : 10.1 sec w/ GeForce GTX 1080 Ti <br>

In [3]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

cuda


### Download Penn Tree Bank (the tensor train_data should consists of 20 columns of ~50,000 words)

In [4]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [5]:
bs = 20

vocab_size = 10000


### Make a recurrent net class

In [6]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding(vocab_size, hidden_size) # COMPLETE HERE
        self.layer2 = nn.LSTM(hidden_size, hidden_size) # COMPLETE HERE
        self.layer3 = nn.Linear(hidden_size, vocab_size) # COMPLETE HERE

        
    def forward(self, word_seq, h_init, c_init ):
        
        g_seq                      = self.layer1(word_seq)  # COMPLETE HERE
        h_seq , (h_final,c_final)  = self.layer2(g_seq, (h_init, c_init))  # COMPLETE HERE (don't forget the extra parenthesis around h_init and c_init)
        score_seq                  = self.layer3(h_seq)  # COMPLETE HERE
        
        return score_seq,  h_final , c_final


### Build the net. Choose the hidden size to be 300. How many parameters in total?

In [7]:
hidden_size= 300 # COMPLETE HERE

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 300)
  (layer2): LSTM(300, 300)
  (layer3): Linear(in_features=300, out_features=10000, bias=True)
)
There are 6732400 (6.73 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [8]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [9]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 5
* sequence length = 35

In [10]:
criterion = nn.CrossEntropyLoss()

my_lr = 5 # COMPLETE HERE

seq_length = 35 # COMPLETE HERE

### Function to evaluate the network on the test set

In [11]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)
   
    h=h.to(device)
    c=c.to(device)
       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h, c  = net( minibatch_data, h , c)
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
        c=c.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 8 passes through the training set.

In [13]:
start=time.time()

for epoch in range(8):
    
    # keep the learning rate to 1 during the first 2 epochs, then divide by 3 at every epoch    
    if epoch >= 2:
        my_lr /= 3 # COMPLETE HERE
        
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h and c to be the zero vector
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)

    # send them to the gpu    
    h=h.to(device)
    c=c.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data = train_data[count:count+seq_length]  # COMPLETE HERE
        minibatch_label = train_data[count+1:count+seq_length+1] # COMPLETE HERE      
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h= h.detach() # COMPLETE HERE
        c= c.detach() # COMPLETE HERE
        h= h.requires_grad_() # COMPLETE HERE
        c= c.requires_grad_() # COMPLETE HERE
                       
        # forward the minibatch through the net        
        scores, h, c  = net(minibatch_data, h, c) # COMPLETE HERE
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          = scores.view(bs*seq_length, vocab_size) # COMPLETE HERE          
        minibatch_label = minibatch_label.view(bs*seq_length) # COMPLETE HERE       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(scores, minibatch_label) # COMPLETE HERE
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward() # COMPLETE HERE

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        # COMPLETE HERE
        optimizer.step()
        
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 13.859039545059204 	 lr= 5 	 exp(loss)= 279.30923092077876
test: exp(loss) =  175.8821561660358

epoch= 1 	 time= 27.62993597984314 	 lr= 5 	 exp(loss)= 127.21404162187932
test: exp(loss) =  133.07444845216824

epoch= 2 	 time= 41.472540616989136 	 lr= 1.6666666666666667 	 exp(loss)= 81.42326689885412
test: exp(loss) =  114.34703385619547

epoch= 3 	 time= 55.45258092880249 	 lr= 0.5555555555555556 	 exp(loss)= 67.17315751614417
test: exp(loss) =  110.29631800506691

epoch= 4 	 time= 69.39259648323059 	 lr= 0.1851851851851852 	 exp(loss)= 62.31148801388744
test: exp(loss) =  108.86604509969871

epoch= 5 	 time= 83.29249620437622 	 lr= 0.0617283950617284 	 exp(loss)= 60.56794418479719
test: exp(loss) =  108.05161133241106

epoch= 6 	 time= 97.58677434921265 	 lr= 0.0205761316872428 	 exp(loss)= 59.935870336821864
test: exp(loss) =  107.53959689298134

epoch= 7 	 time= 112.64601874351501 	 lr= 0.006858710562414266 	 exp(loss)= 59.703638866349245
test: exp(loss) =  107.2

### Choose one sentence (taken from the test set)

In [14]:
sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# or make your own sentence.  No capital letter or punctuation allowed. Each word must be in the allowed vocabulary.
sentence6= "he was very"

# SELECT THE SENTENCE HERE
mysentence = sentence3

### Convert the sentence into a vector, then send to GPU

In [15]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[1786],
        [8705],
        [3246],
        [ 416],
        [  27],
        [  35],
        [2664],
        [ 209],
        [ 108],
        [  32],
        [3017]], device='cuda:0')


### Set the initial hidden state to zero, then run the LSTM.

In [16]:
h = torch.zeros(1, 1, hidden_size)
c = torch.zeros(1, 1, hidden_size)
h=h.to(device)
c=c.to(device)

scores , h, c = net(minibatch_data , h, c)

### Display the network prediction for the next word

In [17]:
print(mysentence, '... \n')

utils.show_next_word(scores)

prices averaging roughly $ N a barrel higher in the third ... 

87.8%	 quarter
1.6%	 period
1.4%	 world
1.1%	 consecutive
0.7%	 area
0.6%	 and
0.5%	 hour
0.4%	 of
0.3%	 quarters
0.2%	 month
0.2%	 <eos>
0.2%	 range
0.2%	 week
0.2%	 game
0.2%	 session
0.2%	 term
0.1%	 day
0.1%	 fiscal
0.1%	 <unk>
0.1%	 1970s
0.1%	 year
0.1%	 bridge
0.1%	 season
0.1%	 hand
0.1%	 half
0.1%	 line
0.1%	 's
0.1%	 floor
0.1%	 &
0.1%	 sector
